<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 검색 증강 생성을 통한 그룹 채팅

자동 생성은 자동화된 채팅을 통해 공동으로 작업을 수행하는 LLM, 툴 또는 사람으로 구동되는 대화형 에이전트를 지원합니다. 이 프레임워크는 다중 에이전트 대화를 통해 툴 사용과 사람의 참여를 허용합니다.
이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)에서 확인할 수 있습니다.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 설치하세요:
```bash
pip 설치 pyautogen
```

In [1]:
%%capture --no-stderr
# %pip install pyautogen[retrievechat]~=0.1.11

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.

In [3]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

LLM models:  ['gpt-35-turbo', 'gpt-35-turbo-0613']


먼저 유효한 json 문자열이어야 하는 환경 변수 "OAI_CONFIG_LIST"를 찾습니다. 해당 변수를 찾을 수 없으면 "OAI_CONFIG_LIST"라는 이름의 json 파일을 찾습니다. 이 파일은 모델별로 구성을 필터링합니다(다른 키로도 필터링할 수 있음).

구성 목록은 다음과 같습니다:
```python
config_list = [
    {
        "모델": "gpt-4",
        "api_key": "<귀하의 OpenAI API 키>",
    }, # gpt-4용 OpenAI API 엔드포인트
    {
        "엔진": "gpt-35-turbo-0631",
        "model": "gpt-35-turbo-0631", # 함수를 사용하려면 0631 이상이 필요합니다.
        "api_base": "<귀하의 Azure OpenAI API 베이스>",
        "api_type": "azure",
        "api_version": "2023-07-01-preview", # 함수를 사용하려면 2023-07-01-preview 이상이 필요합니다.
        "api_key": "<귀하의 Azure OpenAI API 키>"
    }
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 "파일 업로드" 아이콘을 선택해 파일을 업로드할 수 있습니다.

YAML 파일에서 불러오기 등 원하는 다른 방법으로 config_list의 값을 설정할 수 있습니다.

## 에이전트 생성

In [4]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb

llm_config = {
    "request_timeout": 60,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="TERMINATE",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

coder = AssistantAgent(
    name="Senior_Python_Engineer",
    is_termination_msg=termination_msg,
    system_message="You are a senior python engineer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

reviewer = autogen.AssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a code reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

PROBLEM = "How to use spark for parallel training in FLAML? Give me sample code."

def _reset_agents():
    boss.reset()
    boss_aid.reset()
    coder.reset()
    pm.reset()
    reviewer.reset()

def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss_aid, coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=3,
    )

def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss, coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )

def call_rag_chat():
    _reset_agents()
    # In this case, we will have multiple user proxy agents and we don't initiate the chat
    # with RAG user proxy agent.
    # In order to use RAG user proxy agent, we need to wrap RAG agents in a function and call
    # it from other agents.
    def retrieve_content(message, n_results=3):
        boss_aid.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = boss_aid._check_update_context(message)
        if (update_context_case1 or update_context_case2) and boss_aid.update_context:
            boss_aid.problem = message if not hasattr(boss_aid, "problem") else boss_aid.problem
            _, ret_msg = boss_aid._generate_retrieve_user_reply(message)
        else:
            ret_msg = boss_aid.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message
    
    boss_aid.human_input_mode = "NEVER" # Disable human input for boss_aid since it only retrieves content.
    
    llm_config = {
        "functions": [
            {
                "name": "retrieve_content",
                "description": "retrieve content for code generation and question answering.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
                        }
                    },
                    "required": ["message"],
                },
            },
        ],
        "config_list": config_list,
        "request_timeout": 60,
        "seed": 42,
    }

    for agent in [coder, pm, reviewer]:
        # update llm_config for assistant agents.
        agent.llm_config.update(llm_config)

    for agent in [boss, coder, pm, reviewer]:
        # register functions for all agents.
        agent.register_function(
            function_map={
                "retrieve_content": retrieve_content,
            }
        )

    groupchat = autogen.GroupChat(
        agents=[boss, coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )

## 채팅 시작

### UserProxyAgent가 올바른 코드를 얻지 못함
[FLAML](https://github.com/microsoft/FLAML)은 2020년에 오픈소스로 공개되었기 때문에 ChatGPT는 이에 익숙합니다. 하지만 Spark 관련 API는 2022년에 추가되었기 때문에 ChatGPT의 학습 데이터에 포함되지 않았습니다. 결과적으로 유효하지 않은 코드가 생성됩니다.

In [5]:
norag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
Senior_Python_Engineer (to chat_manager):

To use Spark for parallel training in FLAML, you can use the `SparkTrials` class provided by FLAML. Here is a sample code:

```python
from flaml import AutoML
from flaml.data import load_credit
from flaml.model import SparkTrials

# Load data
X_train, y_train, X_test, y_test = load_credit()

# Define the search space
search_space = {
    "n_estimators": {"domain": range(10, 100)},
    "max_depth": {"domain": range(6, 10)},
    "learning_rate": {"domain": (0.01, 0.1, 1)},
}

# Create an AutoML instance with SparkTrials
automl = AutoML(
    search_space=search_space,
    task="classification",
    n_jobs=1,
    ensemble_si

### RetrieveUserProxyAgent가 올바른 코드를 가져옵니다.
RetrieveUserProxyAgent는 주어진 문서 파일을 기반으로 검색 증강 생성을 수행할 수 있으므로 ChatGPT가 올바른 코드를 생성할 수 있습니다!

In [6]:
rag_chat()
# type exit to terminate the chat

Trying to create collection.


INFO:autogen.retrieve_utils:Found 2 chunks.


doc_ids:  [['doc_0', 'doc_1', 'doc_4']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Adding doc_id doc_4 to context.
Boss_Assistant (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to use spark for parallel training in FLAML? Give me sample code.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:
- Use Spark ML estimators for AutoML.
- Use Spark to run training in parallel spark jobs.

## Spark M

다른 사용자 프록시 에이전트와 채팅을 초기화하는 동안 ### RetrieveUserProxyAgent 호출하기
때때로 그룹 채팅에서 채팅을 초기화하지 않고 RetrieveUserProxyAgent를 사용해야 하는 경우가 있을 수 있습니다. 이러한 시나리오에서는 RAG 에이전트를 래핑하여 다른 에이전트에서 호출할 수 있도록 하는 함수를 만드는 것이 필수적입니다.

In [7]:
call_rag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
Senior_Python_Engineer (to chat_manager):

***** Suggested function Call: retrieve_content *****
Arguments: 
{
  "message": "How to use spark for parallel training in FLAML?"
}
*****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION retrieve_content...
doc_ids:  [['doc_0', 'doc_1', 'doc_4']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Adding doc_id doc_4 to context.
Senior_Python_Engineer (to chat_manager):

***** Response from calling function "retrieve_content" *****
You're a retrieve augmented coding assistant. You answer user's ques